In [6]:
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
import skl2onnx
import sklearn
import sklearn.datasets

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
housing

df = pd.DataFrame(data=np.c_[housing['data'], housing['target']], columns=housing['feature_names'] + ['MEDV'])
 
target_column = 'MEDV'
 
# Split the data frame into features and target
x_train = pd.DataFrame(df.drop([target_column], axis = 1))
y_train = pd.DataFrame(df.iloc[:,df.columns.tolist().index(target_column)])

print("\n*** Training dataset x\n")
print(x_train.head())

print("\n*** Training dataset y\n")
print(y_train.head())


*** Training dataset x

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  

*** Training dataset y

    MEDV
0  4.526
1  3.585
2  3.521
3  3.413
4  3.422


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7])])),
                ('regressor', LinearRegression())])

In [8]:
# Score the model
from sklearn.metrics import r2_score, mean_squared_error
y_pred = model.predict(x_train)
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.606232685199805
*** Scikit-learn MSE: 0.5243209861846072


In [9]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

In [10]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(model, 'Cali Data', convert_dataframe_schema(x_train), final_types=[('variable1',FloatTensorType([1,1]))])
# Save the onnx model locally
onnx_model_path = 'cali1.model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)

In [11]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model_path)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = r2_score(y_train, y_pred)
onnx_mse = mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print('R2 Scores are equal' if sklearn_r2_score == onnx_r2_score else 'Difference in R2 scores: {}'.format(abs(sklearn_r2_score - onnx_r2_score)))
print('MSE are equal' if sklearn_mse == onnx_mse else 'Difference in MSE scores: {}'.format(abs(sklearn_mse - onnx_mse)))
print()


*** Onnx r2 score: 0.6062326851701203
*** Onnx MSE: 0.5243209862241339

Difference in R2 scores: 2.9684699143217586e-11
Difference in MSE scores: 3.9526715234217136e-11



In [ ]:
##
# CREATE THE MODEL TABLE IN YOUR DEDICATED SQL POOL
# -- Sample table schema for storing a model and related data
# CREATE TABLE [dbo].[Models]
# (
#    [Id] [int] IDENTITY(1,1) NOT NULL,
#    [Model] [varbinary](max) NULL,
#    [Description] [varchar](200) NULL
# )
# WITH
# (
#    DISTRIBUTION = ROUND_ROBIN,
#    HEAP
# )
# GO
##

In [32]:
import pyodbc

server = 'knoxsynapse.sql.azuresynapse.net' # Azure Synapse Dedicated Pool URL
username = 'knox@knoxsdata.com' # SQL Server username
password = '' # SQL Server password
database = 'knoxdw' # Synapse Database Name
table_name = 'Models' # Table create in prev step
driver='{ODBC Driver 17 for SQL Server}'

# Connect to db database
db_connection_string = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password+';Authentication=ActiveDirectoryPassword'

conn = pyodbc.connect(db_connection_string)
cursor = conn.cursor()

# Insert the ONNX model into the models table
query = f"insert into {table_name} ([description], [model]) values ('Onnx Model',?)"

model_bits = onnx_model.SerializeToString()

insert_params  = (pyodbc.Binary(model_bits))
cursor.execute(query, insert_params)
conn.commit()

In [49]:
import sqlalchemy
from sqlalchemy import create_engine
import urllib


server = 'knoxsynapse.sql.azuresynapse.net' # Azure Synapse Dedicated Pool URL
username = 'knox@knoxsdata.com' # SQL Server username
password = '' # SQL Server password
database = 'knoxdw' # Synapse Database Name
table_name = 'Models' # Table create in prev step
driver='{ODBC Driver 17 for SQL Server}'

# Connect to db database
db_connection_string = 'DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+password+';Authentication=ActiveDirectoryPassword'

conn = pyodbc.connect(db_connection_string)
conn.autocommit = True
cursor = conn.cursor()

features_table_name = 'features'

# Create the features table
query = \
    f'create table {features_table_name} ( ' \
    f' [MedInc] float, [HouseAge] float, [AveRooms] float, [AveBedrms] float, [Population] float, [AveOccup] float, [Latitude] float, [Longitude] float, [id] int)'

cursor.execute(query)
conn.commit()

# Target table

target_table_name = 'target'



# Create the target table
query = \
    f'create table {target_table_name} ( ' \
    f'    [MEDV] float, ' \
    f'    [id] int)'
cursor.execute(query)
conn.commit()

x_train['id'] = range(1, len(x_train)+1)
y_train['id'] = range(1, len(y_train)+1)

print(x_train.head())
print(y_train.head())

   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  id  
0    -122.23   1  
1    -122.22   2  
2    -122.24   3  
3    -122.25   4  
4    -122.25   5  
    MEDV  id
0  4.526   1
1  3.585   2
2  3.521   3
3  3.413   4
4  3.422   5


In [55]:
x_train.to_csv('features.csv')
y_train.to_csv('target.csv')

In [ ]:
COPY INTO dbo.target
FROM 'https://knoxlakegen2.blob.core.windows.net/etlload/target.csv'
WITH (FILE_TYPE='csv',FIRSTROW=2)

COPY INTO dbo.features
FROM 'https://knoxlakegen2.blob.core.windows.net/etlload/features.csv'
WITH (FILE_TYPE='csv',FIRSTROW=2)

In [ ]:
### QUERY YOUR MODELS TABLE TO GET YOUR MODEL ID
DECLARE @model VARBINARY(max) = (SELECT Model FROM models WHERE id = 59);

SELECT d.*, p.variable1 AS SCORE
FROM PREDICT(MODEL = @model,
    DATA = dbo.features AS d, RUNTIME = ONNX) WITH (variable1 FLOAT) AS p;